In [6]:
from op_analytics.datasources.chainsrpc.erc20metadata import get_rpc_for_chain, execute_pull

get_rpc_for_chain(10)

'https://mainnet.optimism.io'

In [7]:
raw_result = execute_pull()

2025-02-13 22:30:36 [info     ] fetched token metadata for TokenContract(chain='swan', chain_id=254, contract_address='0xbb4ec1b56cb624863298740fd264ef2f910d5564') counter=004/010 eta=None filename=erc20metadata.py lineno=149 process=52591
2025-02-13 22:30:36 [info     ] fetched token metadata for TokenContract(chain='mint', chain_id=185, contract_address='0x63cffa2af25090d850d0ffc23a0f57a41cdcfc3e') counter=005/010 eta=None filename=erc20metadata.py lineno=149 process=52591
2025-02-13 22:30:36 [info     ] fetched token metadata for TokenContract(chain='cyber', chain_id=7560, contract_address='0x14778860e937f509e651192a90589de711fb88a9') counter=001/010 eta=None filename=erc20metadata.py lineno=149 process=52591
2025-02-13 22:30:36 [info     ] fetched token metadata for TokenContract(chain='mint', chain_id=185, contract_address='0x05d032ac25d322df992303dca074ee7392c117b9') counter=006/010 eta=None filename=erc20metadata.py lineno=149 process=52591
2025-02-13 22:30:36 [info     ] fetche

In [3]:
raw_result

{TokenContract(chain='swan', chain_id=254, contract_address='0xbb4ec1b56cb624863298740fd264ef2f910d5564'): {'chain': 'swan',
  'chain_id': 254,
  'contract_address': '0xbb4ec1b56cb624863298740fd264ef2f910d5564',
  'decimals': 18,
  'symbol': 'SWAN',
  'name': 'Swan Token',
  'total_supply': 92363045558845532272951750,
  'block_number': 4172693,
  'block_timestamp': 1739503680},
 TokenContract(chain='mint', chain_id=185, contract_address='0x63cffa2af25090d850d0ffc23a0f57a41cdcfc3e'): {'chain': 'mint',
  'chain_id': 185,
  'contract_address': '0x63cffa2af25090d850d0ffc23a0f57a41cdcfc3e',
  'decimals': 18,
  'symbol': 'Margarita ',
  'name': 'Margarita ',
  'total_supply': 157077478000000000000000000,
  'block_number': 11947376,
  'block_timestamp': 1739503683},
 TokenContract(chain='mint', chain_id=185, contract_address='0x05d032ac25d322df992303dca074ee7392c117b9'): {'chain': 'mint',
  'chain_id': 185,
  'contract_address': '0x05d032ac25d322df992303dca074ee7392c117b9',
  'decimals': 6,
 

In [5]:
import polars as pl
metadata_rows = list(raw_result.values())

df = pl.DataFrame(metadata_rows)
df

chain,chain_id,contract_address,decimals,symbol,name,total_supply,block_number,block_timestamp
str,i64,str,i64,str,str,i128,i64,i64
"""swan""",254,"""0xbb4ec1b56cb624863298740fd264…",18,"""SWAN""","""Swan Token""",92363045558845532272951750,4172693,1739503680
"""mint""",185,"""0x63cffa2af25090d850d0ffc23a0f…",18,"""Margarita ""","""Margarita """,157077478000000000000000000,11947376,1739503683
"""mint""",185,"""0x05d032ac25d322df992303dca074…",6,"""USDT""","""Tether USD""",200221347794,11947376,1739503683
"""cyber""",7560,"""0x14778860e937f509e651192a9058…",18,"""CYBER""","""CyberConnect""",7398611579399000000000000,13037557,1739503681
"""mint""",185,"""0x1a08672d34845d254adc2d3b75ba…",18,"""potenza555""","""potenza555""",102545211000000000000000000,11947376,1739503683
"""mint""",185,"""0x15cb30c538bff5b90a6b0b9a6774…",18,"""DDK""","""Dodik""",183691990000000000000000000,11947376,1739503683
"""mint""",185,"""0x1ebeafd5ce947267dbf1694cc089…",18,"""Holdaran's""","""Holdara""",177892582000000000000000000,11947376,1739503683
"""mint""",185,"""0x272e7d0ac2332ee23ab6f2f1e111…",18,"""BRN""","""brown""",181301557000000000000000000,11947376,1739503683
"""kroma""",255,"""0x0257e4d92c00c9efcca1d641b224…",6,"""USDC""","""USD Coin""",302527594656,22811648,1739503683


In [48]:
def decode_abi_encoded_string(hex_str: str) -> str:
    """
    Decode an ABI-encoded dynamic string that comes back 
    from an eth_call returning (string).
    """
    # Strip off 0x prefix, if present
    if hex_str.startswith("0x"):
        hex_str = hex_str[2:]
    
    # Convert remaining hex to bytes
    data = bytes.fromhex(hex_str)
    
    # The next 32 bytes are the string length
    str_len = int.from_bytes(data[32:64], byteorder='big')
    
    # The next str_len bytes are the actual UTF-8 bytes of the string
    string_bytes = data[64:64 + str_len]
    
    return string_bytes.decode('utf-8', errors='replace')

In [ ]:
import requests

RPC_URL = 'https://mainnet.base.org/'

CONTRACT_ADDRESS = "0x833589fcd6edb6e08f4c7c32d4f71b54bda02913"

METHOD_IDS = {
    "decimals": "0x313ce567",
    "symbol": "0x95d89b41",
    "name": "0x06fdde03",
    "totalSupply": "0x18160ddd"
}

# Construct JSON-RPC payloads for contract calls + block queries
payloads = []
request_id = 1

# 1) Add ERC20 method calls
for method_id in METHOD_IDS.values():
    payloads.append({
        "jsonrpc": "2.0",
        "method": "eth_call",
        "params": [{
            "to": CONTRACT_ADDRESS,
            "data": method_id
        }, "latest"],
        "id": request_id
    })
    request_id += 1

# 2) Add a request for current block number
payloads.append({
    "jsonrpc": "2.0",
    "method": "eth_blockNumber",
    "params": [],
    "id": request_id
})
request_id += 1

# 3) Add a request for the latest block details
payloads.append({
    "jsonrpc": "2.0",
    "method": "eth_getBlockByNumber",
    "params": ["latest", False],
    "id": request_id
})
request_id += 1

# Send requests
response = requests.post(RPC_URL, json=payloads)
results = response.json()

# Parse ERC20 metadata responses
decimals = int(results[0]["result"], 16)
symbol = decode_abi_encoded_string(results[1]["result"])
name = decode_abi_encoded_string(results[2]["result"])
total_supply = int(results[3]["result"], 16)
block_number_hex = results[4]["result"]
latest_block_data = results[5]["result"]

block_number = int(block_number_hex, 16)
block_timestamp = int(latest_block_data["timestamp"], 16)

result = {
    "block_number": block_number,
    "block_timestamp": block_timestamp,
    "contract_address": CONTRACT_ADDRESS,
    "decimals": decimals,
    "symbol": symbol,
    "name": name,
    "total_supply": total_supply
}

result

{'block_number': 26355559,
 'block_timestamp': 1739500465,
 'contract_address': '0x833589fcd6edb6e08f4c7c32d4f71b54bda02913',
 'decimals': 6,
 'symbol': 'USDC',
 'name': 'USD Coin',
 'total_supply': 3635635139906944}

In [3]:
import pandas as pd
import sys
sys.path.append("../helper_functions")
from web3 import Web3
from datetime import datetime, timezone
sys.path.pop()
import re

In [4]:
def clean_string(raw_str: str) -> str:
    """
    Remove all control characters (ASCII 0–31, plus 127),
    then strip leading/trailing whitespace.
    """
    return re.sub(r'[\x00-\x1F\x7F]+', '', raw_str).strip()


def get_token_metadata(token_contract_address, rpc, method_name_list):
    """
    Aggregates multiple contract calls into one Python function call.
    (Still multiple calls to your Ethereum node, but simpler to manage in code.)
    """
    w3_conn = Web3(Web3.HTTPProvider(rpc))
    token_contract_address = token_contract_address.lower() 
    
    block_timestamp = w3_conn.eth.get_block('latest').timestamp
    block_number = w3_conn.eth.get_block('latest').number
    # Convert the UNIX timestamp to a human-readable format
    block_datetime = datetime.fromtimestamp(block_timestamp, tz=timezone.utc)
    block_time = block_datetime.strftime('%Y-%m-%d %H:%M:%S')

    method_ids = [Web3.keccak(text=f"{method_name}()")[:4].hex() for method_name in method_name_list]
    
    # create a loop to get the response for each method
    response_map = {}

    for method_id, method_name in zip(method_ids, method_name_list):
        response = w3_conn.eth.call({
            'to': token_contract_address,
            'data': method_id
        })
        response_map[method_name] = response


    # return the response for each method
    
    decimals = Web3.to_int(hexstr=response_map["decimals"].hex())
    symbol = clean_string(Web3.to_text(hexstr=response_map["symbol"].hex()))
    name = clean_string(Web3.to_text(hexstr=response_map["name"].hex()))
    total_supply = Web3.to_int(hexstr=response_map["totalSupply"].hex())
    
    return {
        "block_time": block_time,
        "block_number": block_number,
        "block_timestamp": block_timestamp,
        "contract_address": token_contract_address,
        "decimals": decimals,
        "symbol": symbol,
        "name": name,
        "total_supply": total_supply
    }

In [5]:
chains_rpcs = pd.read_csv('../../op_chains_tracking/outputs/chain_metadata.csv', na_filter=False)
chains_rpcs = chains_rpcs[~(chains_rpcs['rpc_url'] == '')]

rpc =chains_rpcs[chains_rpcs['chain_name'] == 'op']["rpc_url"]
RPC = rpc.iloc[0] 

CONTRACT_ADDRESS = "0x4200000000000000000000000000000000000006"
METHOD_NAME = ["decimals", "symbol", "name", "totalSupply"]


In [14]:
response = get_token_metadata(CONTRACT_ADDRESS, RPC, METHOD_NAME)
response

{'block_time': '2025-02-13 21:06:07',
 'block_number': 131940995,
 'block_timestamp': 1739480767,
 'contract_address': '0x4200000000000000000000000000000000000006',
 'decimals': 18,
 'symbol': 'WETH',
 'name': 'Wrapped Ether',
 'total_supply': 38885309302187337741110}